## Introduzione
Questi due blocchi di codice lavorano insieme per elaborare e analizzare i dati delle carriere dei giocatori NBA. Il primo codice scarica i dati dei giocatori e li prepara, salvandoli in un file CSV con informazioni come ID del giocatore, nome e stagioni attive.

Il secondo codice utilizza questi dati per analizzare le performance di ciascun giocatore. Recupera le statistiche delle partite stagionali da un'API, calcola metriche avanzate (come percentuali di tiro e medie per partita), e aggrega il tutto in un nuovo file CSV (**`nba_stats.csv`**
) pronto per analisi approfondite o visualizzazioni.

Questa struttura consente di andare dal download e pulizia dei dati grezzi fino alla generazione di statistiche avanzate per un'intera carriera.

#### Installazione delle librerie necessarie
Installa la libreria nba_api per scaricare e lavorare con dataset relativi alla NBA.

In [1]:
!pip install nba_api

#### PRIMO CODICE
#### Importazione delle librerie e scaricamento dei dati
Questa cella importa le librerie necessarie e scarica i dati dei giocatori NBA utilizzando l'endpoint CommonAllPlayers.

In [4]:
import pandas as pd
from nba_api.stats.endpoints import commonallplayers

# Scarica i dati dei giocatori NBA
all_players = commonallplayers.CommonAllPlayers().get_data_frames()[0]

#### Pulizia e filtraggio dei dati
Convertiamo le colonne 'FROM_YEAR' e 'TO_YEAR' in tipo numerico, gestendo eventuali valori mancanti, e filtriamo i giocatori che hanno iniziato la loro carriera dopo la stagione 1980-81, rimuovendo eventuali anomalie.

In [5]:
# Converte le colonne 'FROM_YEAR' e 'TO_YEAR' in tipo int64
all_players['FROM_YEAR'] = pd.to_numeric(all_players['FROM_YEAR'], errors='coerce').fillna(0).astype('int64')
all_players['TO_YEAR'] = pd.to_numeric(all_players['TO_YEAR'], errors='coerce').fillna(0).astype('int64')

# Filtra i giocatori che hanno iniziato la loro carriera dopo la stagione 2000-01
all_players = all_players[(all_players['FROM_YEAR'] > 1980) & 
                          (all_players['FROM_YEAR'] != 2024) & 
                          (all_players['FROM_YEAR'] < all_players['TO_YEAR'])]

#### Creazione e formattazione del DataFrame dei giocatori
Creiamo un DataFrame con i dati unici dei giocatori, rinominiamo le colonne per chiarezza, modifichiamo il formato delle stagioni e selezioniamo i primi 30 giocatori.

In [6]:
# Crea un DataFrame con ID, nome e anni di carriera dei giocatori unici
unique_players = all_players[['PERSON_ID', 'DISPLAY_FIRST_LAST', 'FROM_YEAR', 'TO_YEAR']].drop_duplicates()

# Rinomina le colonne per chiarezza
unique_players.columns = ['Player_ID', 'Player_Name', 'First_Season', 'Last_Season']

# Modifica il formato delle stagioni
unique_players['First_Season'] = unique_players['First_Season'].astype(str) + '-' + unique_players['First_Season'].add(1).astype(str).str[-2:]
unique_players['Last_Season'] = (unique_players['Last_Season'] - 1).astype(str) + '-' + unique_players['Last_Season'].astype(str).str[-2:]

# Filtra solo i primi 30 giocatori
unique_players = unique_players.head(30)


#### Salvataggio dei dati in un file CSV
In questa cella salviamo il DataFrame nel file player_nba.csv.

In [7]:
# Salva il DataFrame in un file CSV
unique_players.to_csv('player_nba.csv', index=False)

print('Dati dei giocatori salvati in: player_nba.csv')


Dati dei giocatori salvati in: player_nba.csv


In [8]:
display(unique_players.head())

,Player_ID,Player_Name,First_Season,Last_Season
0,76001,Alaa Abdelnaby,1990-91,1993-94
3,51,Mahmoud Abdul-Rauf,1990-91,1999-00
4,1505,Tariq Abdul-Wahad,1997-98,2002-03
5,949,Shareef Abdur-Rahim,1996-97,2006-07
9,203518,Alex Abrines,2016-17,2017-18


#### SECONDO CODICE
#### Importazione delle librerie e caricamento dati
In questa cella importiamo le librerie necessarie, carichiamo il file CSV dei giocatori e generiamo le stagioni dal 2000-01 all'anno corrente.

In [9]:
import pandas as pd
import numpy as np
from nba_api.stats.endpoints import playergamelog
from tqdm import tqdm  # Barra di avanzamento
from datetime import datetime

# Carica il CSV dei giocatori
players_df = pd.read_csv('player_nba.csv')

# Genera le stagioni da 2000-01 all'anno corrente
start_year = 2000
current_year = datetime.now().year
seasons = [f"{year}-{str(year + 1)[-2:]}" for year in range(start_year, current_year + 1)]


#### Elaborazione dei dati per ciascun giocatore
Ciclo attraverso i giocatori per ottenere le loro statistiche complessive. Utilizziamo una barra di avanzamento per monitorare il progresso.

In [10]:
# DataFrame per conservare i dati aggregati di tutti i giocatori
all_players_stats = pd.DataFrame()

# Ciclo attraverso i giocatori con barra di avanzamento
for index, row in tqdm(players_df.iterrows(), total=players_df.shape[0], desc="Elaborazione giocatori"):
    player_id = row['Player_ID']
    player_name = row['Player_Name']
    first_season = row['First_Season']
    last_season = row['Last_Season']

    # Filtra le stagioni in base alla prima e ultima stagione del giocatore
    valid_seasons = [season for season in seasons if first_season <= season <= last_season]

    # DataFrame per conservare i dati aggregati per tutte le stagioni del giocatore
    player_stats_total = pd.DataFrame()

    total_minutes = 0  # Per il calcolo della media dei minuti
    total_games = 0    # Per contare il numero di partite giocate

    # Variabili per il calcolo delle statistiche totali
    total_stats = {
        'MIN': 0,
        'FGM': 0,
        'FGA': 0,
        'FG3M': 0,
        'FG3A': 0,
        'FTM': 0,
        'FTA': 0,
        'OREB': 0,
        'DREB': 0,
        'REB': 0,
        'AST': 0,
        'TOV': 0,
        'STL': 0,
        'BLK': 0,
        'PF': 0,
        'PTS': 0,
        'PLUS_MINUS': 0
    }

    # Ciclo attraverso le stagioni valide
    for season in valid_seasons:
        try:
            # Recupero dei dati di gioco per la stagione corrente
            gamelog = playergamelog.PlayerGameLog(player_id=player_id, season=season).get_data_frames()[0]

            if not gamelog.empty:  # Verifica se il gamelog non è vuoto
                # Aggregazione dei dati totali per la stagione
                aggregated_stats = gamelog.sum()

                # Aggiunta dei dati aggregati alle statistiche totali
                total_minutes += aggregated_stats['MIN']
                total_games += len(gamelog)

                for key in total_stats.keys():
                    total_stats[key] += aggregated_stats[key]

        except Exception as e:
            print(f"Errore per {player_name} nella stagione {season}: {e}")

    # Calcolo delle statistiche complessive per il giocatore
    if total_games > 0:
        avg_minutes_per_game = total_minutes / total_games
        FG_PCT = total_stats['FGM'] / total_stats['FGA'] if total_stats['FGA'] != 0 else 0
        FG3_PCT = total_stats['FG3M'] / total_stats['FG3A'] if total_stats['FG3A'] != 0 else 0
        FT_PCT = total_stats['FTM'] / total_stats['FTA'] if total_stats['FTA'] != 0 else 0
        AST_TO_RATIO = total_stats['AST'] / total_stats['TOV'] if total_stats['TOV'] != 0 else 0

        # Crea un dizionario con le statistiche del giocatore
        player_career_stats = {
            'PLAYER_ID': player_id,
            'PLAYER_NAME': player_name,
            'SEASON_YEAR': f"{first_season} - {last_season}",
            'TOTAL_MIN': total_minutes,
            'TOTAL_GAMES': total_games,
            'AVG_MIN': avg_minutes_per_game,
            'FG_PCT': FG_PCT,
            'FG3_PCT': FG3_PCT,
            'FT_PCT': FT_PCT,
            'AST_TO_RATIO': AST_TO_RATIO,
        }

        # Aggiunge le statistiche totali
        for key, value in total_stats.items():
            player_career_stats[key] = value

        # Trasforma il dizionario in DataFrame
        player_career_stats_df = pd.DataFrame([player_career_stats])

        # Aggiunge i dati aggregati al DataFrame principale
        all_players_stats = pd.concat([all_players_stats, player_career_stats_df], ignore_index=True)


Elaborazione giocatori: 100%|███████████████████| 30/30 [01:55<00:00,  3.85s/it]


#### Calcolo delle statistiche avanzate
In questa cella calcoliamo statistiche derivate come FG2 (field goals a due punti), FG2_PCT (percentuale) e altre per partita.

In [11]:
# Aggiungi le colonne FGM2 e FG2A
all_players_stats['FG2M'] = all_players_stats['FGM'] - all_players_stats['FG3M']
all_players_stats['FG2A'] = all_players_stats['FGA'] - all_players_stats['FG3A']

# Statistiche da mediare
stats_to_average = [
    'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 
    'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 
    'BLK', 'PF', 'PTS', 'FG2M', 'FG2A'
]

# Calcola le statistiche per partita
for stat in stats_to_average:
    all_players_stats[f'{stat}_PER_GAME'] = all_players_stats[stat] / all_players_stats['TOTAL_GAMES']

# Calcola FG2_PCT e eFG_PCT
all_players_stats['FG2_PCT'] = np.where(all_players_stats['FG2A'] > 0, all_players_stats['FG2M'] / all_players_stats['FG2A'], np.nan)
all_players_stats['eFG_PCT'] = (all_players_stats['FGM'] + 0.5 * all_players_stats['FG3M']) / all_players_stats['FGA']

#### Riorganizzazione e salvataggio del DataFrame
Riorganizziamo le colonne secondo un ordine specifico e salviamo il risultato finale in un file CSV.

In [12]:
# Riorganizza le colonne secondo l'ordine specificato
ordered_columns = [
    'PLAYER_ID', 'PLAYER_NAME', 'SEASON_YEAR', 'TOTAL_MIN', 'TOTAL_GAMES', 'AVG_MIN',
    'FGM', 'FGA', 'FG_PCT', 'FGM_PER_GAME', 'FGA_PER_GAME',
    'FG2M', 'FG2A', 'FG2_PCT', 'FG2M_PER_GAME', 'FG2A_PER_GAME',
    'FG3M', 'FG3A', 'FG3_PCT', 'FG3M_PER_GAME', 'FG3A_PER_GAME', 'eFG_PCT',
    'FTM', 'FTA', 'FT_PCT', 'FTM_PER_GAME', 'FTA_PER_GAME',
    'OREB', 'DREB', 'REB', 'OREB_PER_GAME', 'DREB_PER_GAME', 'REB_PER_GAME',
    'AST', 'TOV', 'AST_TO_RATIO', 'AST_PER_GAME', 'TOV_PER_GAME',
    'STL', 'BLK', 'STL_PER_GAME', 'BLK_PER_GAME',
    'PF', 'PF_PER_GAME',
    'PTS', 'PTS_PER_GAME', 'PLUS_MINUS'
]

# Riorganizza il DataFrame con l'ordine specificato
all_players_stats = all_players_stats[ordered_columns]

all_players_stats = all_players_stats[ordered_columns]

# Controllo delle righe duplicate
print(f"Numero di righe prima della rimozione dei duplicati: {len(all_players_stats)}")
all_players_stats = all_players_stats.drop_duplicates()
print(f"Numero di righe dopo la rimozione dei duplicati: {len(all_players_stats)}")

# Salvataggio del nuovo CSV
all_players_stats.to_csv("nba_stats.csv", index=False)
print("Dati aggregati e trasformati salvati in 'nba_stats.csv'")


Numero di righe prima della rimozione dei duplicati: 22
Numero di righe dopo la rimozione dei duplicati: 22
Dati aggregati e trasformati salvati in 'nba_stats.csv'


In [13]:
display(all_players_stats.head())

,PLAYER_ID,PLAYER_NAME,SEASON_YEAR,TOTAL_MIN,TOTAL_GAMES,AVG_MIN,FGM,FGA,FG_PCT,FGM_PER_GAME,...,TOV_PER_GAME,STL,BLK,STL_PER_GAME,BLK_PER_GAME,PF,PF_PER_GAME,PTS,PTS_PER_GAME,PLUS_MINUS
0,1505,Tariq Abdul-Wahad,1997-98 - 2002-03,1066,67,15.910448,125,316,0.395570,1.865672,...,1.014925,40,26,0.597015,0.388060,136,2.029851,303,4.522388,-127
1,949,Shareef Abdur-Rahim,1996-97 - 2006-07,17853,530,33.684906,3248,6770,0.479764,6.128302,...,2.298113,493,341,0.930189,0.643396,1534,2.894340,8880,16.754717,-1177
2,203518,Alex Abrines,2016-17 - 2017-18,2190,143,15.314685,249,632,0.393987,1.741259,...,0.405594,75,16,0.524476,0.111888,238,1.664336,759,5.307692,29
3,1630173,Precious Achiuwa,2020-21 - 2023-24,5233,263,19.897338,820,1704,0.481221,3.117871,...,1.022814,134,167,0.509506,0.634981,487,1.851711,2041,7.760456,-145
4,101165,Alex Acker,2005-06 - 2007-08,36,5,7.200000,4,16,0.250000,0.800000,...,0.800000,1,0,0.200000,0.000000,4,0.800000,9,1.800000,-33


### PASSO FINALE
Nei passaggi precedenti, abbiamo elaborato i dati sulle carriere e le statistiche dei giocatori NBA. Ora ci concentriamo sull'etichettatura dei dati per preparare un dataset utilizzabile per il training di un classificatore. Utilizziamo una funzione per assegnare a ogni giocatore un'etichetta (**`playertype`**) basata sul suo stile di gioco. Questo processo è essenziale per definire il target di apprendimento che sarà utilizzato nell'addestramento del modello di machine learning.

In [14]:
import pandas as pd

# Carica il file CSV in un DataFrame
df = pd.read_csv('nba_stats.csv')

#### Definizione della funzione per creare le etichette (player_type)
In questa cella, definiamo la logica che determina l'etichetta di ciascun giocatore in base al suo stile di gioco. Queste etichette rappresentano il target del nostro classificatore supervisionato.

In [15]:
# Funzione per classificare i giocatori
def classify_player(row):
    # Inizializza i flag per ogni tipo di giocatore
    shooter = (
        row['FG3_PCT'] >= 0.35 and
        row['eFG_PCT'] >= 0.50 and
        row['FG3A_PER_GAME'] >= 3
    )
    
    attacker = (
        row['FG2_PCT'] >= 0.44 and
        row['FTA_PER_GAME'] >= 1.5 and
        row['FG2A_PER_GAME'] >= 4.6
    )
    
    assist_man = (
        row['AST_TO_RATIO'] >= 1.2 and
        row['AST_PER_GAME'] >= 3.5
    )
    
    defender_pro = (
        row['STL_PER_GAME'] >= 0.9 or row['BLK_PER_GAME'] >= 0.6
    )
    
    # Conta quante categorie soddisfa il giocatore
    categories = sum([shooter, attacker, assist_man, defender_pro])
    
    # Classifica il giocatore in base ai criteri
    if categories == 0:
        return 7  # ROTATION PLAYER
    elif categories == 1:
        if shooter:
            return 1  # SHOOTER
        elif attacker:
            return 2  # ATTACKER
        elif assist_man:
            return 3  # ASSIST MAN
        elif defender_pro:
            return 4  # DEFENDER PRO
    elif categories == 2:
        return 5  # ROLE_PLAYER
    elif categories >= 3:
        return 6  # NBA STAR


#### Applicazione della funzione e salvataggio del dataset etichettato
Applichiamo la funzione ai dati, generando la colonna player_type.

In [16]:
# Applica la funzione per classificare ciascun giocatore e crea la colonna 'player_type'
df['player_type'] = df.apply(classify_player, axis=1)

# Salva il DataFrame modificato in un nuovo file CSV
df.to_csv('nba_train.csv', index=False)

print("Dataset etichettato salvato in 'nba_train.csv'")


Dataset etichettato salvato in 'nba_train.csv'


#### Conclusione
Abbiamo creato un dataset etichettato che rappresenta il target necessario per il training di un classificatore supervisionato. Le etichette (player_type) categorizzano i giocatori in base al loro stile di gioco, facilitando l'apprendimento automatico. Il file **`nba_data.csv`** è quello utilizzato come input nel processo di training del modello poichè quello creato da questo script è una versione altreattanto dettagliata ma ristretta per facilitare l'esecuzione.

#### Evoluzione del Gioco NBA

In [17]:
import pandas as pd

# Caricamento del CSV
file_path = "nba_data.csv" 
data = pd.read_csv(file_path)

# Funzione per classificare le stagioni
def classify_season(season_year):
    try:
        start_year = int(season_year.split('-')[0])
        if start_year < 2000:
            return 'Anni 80/90'
        else:
            return 'Anni 2000+'
    except Exception as e:
        print(f"Errore nella classificazione della stagione: {season_year}, {e}")
        return 'Unknown'

# Divsione per anno
data['Season_Class'] = data['SEASON_YEAR'].apply(classify_season)

# Raggruppare i dati per Season_Class e player_type, calcolando il conteggio
summary = data.groupby(['Season_Class', 'player_type']).size().reset_index(name='Count')

# Calcolare il totale dei giocatori per ciascun gruppo
totals = summary.groupby('Season_Class')['Count'].sum().reset_index(name='Total')

# Unire i totali con il riepilogo
summary = summary.merge(totals, on='Season_Class')

# Calcolare la percentuale
summary['Percentage'] = (summary['Count'] / summary['Total']) * 100

# Stampare i risultati
print("Distribuzione percentuale dei giocatori per classe (player_type):\n")
for season_class in summary['Season_Class'].unique():
    print(f"\nGiocatori {season_class}:")
    season_data = summary[summary['Season_Class'] == season_class]
    for _, row in season_data.iterrows():
        print(f"Classe {row['player_type']}: {row['Percentage']:.2f}%")


Distribuzione percentuale dei giocatori per classe (player_type):


Giocatori Anni 2000+:
Classe 1: 5.87%
Classe 2: 6.33%
Classe 3: 1.44%
Classe 4: 9.79%
Classe 5: 16.12%
Classe 6: 7.70%
Classe 7: 52.74%

Giocatori Anni 80/90:
Classe 1: 0.50%
Classe 2: 12.90%
Classe 3: 0.79%
Classe 4: 8.83%
Classe 5: 21.53%
Classe 6: 9.03%
Classe 7: 46.43%
